In [1]:
from statsmodels.datasets import presidential2016

ImportError: No module named 'statsmodels'

In [135]:
import random
random.randint(0, 10)

4

In [2]:
import os
import sys


In [ ]:
os.chdir("../..")


In [20]:
os.getcwd()


'/Users/gjlondon/programming/open_source/statsmodels'

In [3]:
# let us import app packages
PACKAGE_PARENT = '/Users/gjlondon/programming/open_source/statsmodels'
sys.path.append(os.path.normpath(PACKAGE_PARENT))

In [4]:
sys.path

['',
 '/usr/local/opt/libxml2/lib/python2.7/site-packages',
 '/Users/gjlondon/programming/open_source/statsmodels/statsmodels/stats',
 '/Users/gjlondon/anaconda/envs/statsmodels-dev/lib/python35.zip',
 '/Users/gjlondon/anaconda/envs/statsmodels-dev/lib/python3.5',
 '/Users/gjlondon/anaconda/envs/statsmodels-dev/lib/python3.5/plat-darwin',
 '/Users/gjlondon/anaconda/envs/statsmodels-dev/lib/python3.5/lib-dynload',
 '/Users/gjlondon/anaconda/envs/statsmodels-dev/lib/python3.5/site-packages',
 '/Users/gjlondon/anaconda/envs/statsmodels-dev/lib/python3.5/site-packages/setuptools-27.2.0-py3.5.egg',
 '/Users/gjlondon/anaconda/envs/statsmodels-dev/lib/python3.5/site-packages/IPython/extensions',
 '/Users/gjlondon/.ipython',
 '/Users/gjlondon/programming/open_source/statsmodels']

In [15]:
import statsmodels.api as sm
from statsmodels.datasets import presidential2016

data = sm.datasets.presidential2016.load_pandas()
data.endog

AttributeError: 'Dataset' object has no attribute 'endog'

In [136]:
data

<class 'statsmodels.datasets.utils.Dataset'>

In [119]:
import pandas as pd
import numpy as np
from functools import partial
import itertools
from numpy import linalg
from scipy.stats import chi2_contingency, chi2


In [130]:
from scipy.stats import chi2_contingency, chi2


class MRCVTable(object):
    """
    Analyses that can be performed on a two-way contingency table that contains
    'multiple response' categorical variables (e.g. 'choose all that apply' questions).

    Parameters
    ----------
    table : array-like
        A contingency table.
    shift_zeros : boolean
        If True and any cell count is zero, add 0.5 to all values
        in the table.

    Attributes
    ----------
    table_orig : array-like
        The original table is cached as `table_orig`.
    marginal_probabilities : tuple of two ndarrays
        The estimated row and column marginal distributions.
    independence_probabilities : ndarray
        Estimated cell probabilities under row/column independence.

    See also
    --------
    statsmodels.graphics.mosaicplot.mosaic
    scipy.stats.chi2_contingency

    Notes
    -----


    References
    ----------
    Definitions of residuals:
        https://onlinecourses.science.psu.edu/stat504/node/86

    Bilder and Loughlin (2004)
    """

    def __init__(self, table, rows_factor, columns_factor, shift_zeros=True):

        self.rows_factor = rows_factor
        self.columns_factor = columns_factor
        self.table = table

        if shift_zeros and (self.table.min() == 0):
            self.table = self.table + 0.5

    def __unicode__(self):
        template = "Contingency Table With Multi-Response Categorical Variables (MRCV's).\nData:\n{table}"
        return template.format(table=self.table)

    def __repr__(self):
        return self.__unicode__()

    @classmethod
    def from_data(cls, data, I, J,
                  rows_factor_name="factor_0", columns_factor_name="factor_1",
                  shift_zeros=True):
        """
        Construct a Table object from data.

        Parameters
        ----------
        data : array-like
            The raw data, from which a contingency table is constructed
            using the first two columns.
        shift_zeros : boolean
            If True and any cell count is zero, add 0.5 to all values
            in the table.
        I: The number of columns in the dataframe corresponding to the first factor
        J: The number of columns in the dataframe corresponding to the second factor

        Returns
        -------
        An MRCVTable instance.
        """

        if isinstance(data, pd.DataFrame):
            rows_data = data.iloc[:, 0:I]
            columns_data = data.iloc[:, I:I + J]
            rows_labels = rows_data.columns
            columns_labels = columns_data.columns
        else:
            rows_data = data[:, 0:I]
            columns_data = data[:, I:I + J]
            rows_labels = ["level_{}".format(i) for i in range(0, I)]
            columns_labels = ["level_{}".format(i) for i in range(I, I + J)]

        rows_factor = Factor(rows_data, labels=rows_labels, name=rows_factor_name, orientation="wide")
        columns_factor = Factor(columns_data, labels=columns_labels, name=columns_factor_name, orientation="wide")
        table = cls.table_from_factors(columns_factor, rows_factor)
        return cls(table, rows_factor, columns_factor, shift_zeros)

    @classmethod
    def table_from_factors(cls, columns_factor, rows_factor):
        row_reshaped = rows_factor.reshape_for_contingency_table()
        col_reshaped = columns_factor.reshape_for_contingency_table()
        joint_dataframe = pd.merge(row_reshaped, col_reshaped, how="inner",
                                   on='observation_number', suffixes=("_row", "_col"))
        # without bool cast, '&' sometimes doesn't know how to compare types
        joint_response = joint_dataframe['value_row'].astype(bool) & joint_dataframe['value_col'].astype(bool)
        joint_dataframe['_joint_response'] = joint_response
        table = pd.pivot_table(joint_dataframe,
                               values='_joint_response',
                               fill_value=0,
                               index=['variable_row'],
                               columns=['variable_col'],
                               aggfunc=np.sum,)
        return table

    def _spmi_stat(self):
        rows_data = self.rows_factor.data
        columns_data = self.columns_factor.data
        rows_levels = self.rows_factor.labels
        columns_levels = self.columns_factor.labels
        chis_spmi = pd.DataFrame(index=rows_levels, columns=columns_levels)
        for i in range(0, len(rows_levels)):
            for j in range(0, len(columns_levels)):
                rows = np.array(rows_data[:, i])
                columns = np.array(columns_data[:, j])
                row_name = rows_levels[i]
                col_name = columns_levels[j]
                crosstab = pd.crosstab(index=rows, columns=columns, rownames=[row_name], colnames=[col_name])
                chi2_results = chi2_contingency(crosstab, correction=False)
                chi_squared_statistic, _, _, _ = chi2_results
                chis_spmi.loc[row_name, col_name] = chi_squared_statistic
        return chis_spmi
    
    def _test_for_single_pairwise_mutual_independence_using_bonferroni(self, observed):
        chi2_survival_with_1_dof = partial(chi2.sf, df=1)
        p_value_ij = observed.applymap(chi2_survival_with_1_dof)
        p_value_min = p_value_ij.min().min()
        bonferroni_correction_factor = self.rows_factor.level_count * self.columns_factor.level_count
        table_p_value_bonferroni_corrected = bonferroni_correction_factor * p_value_min
        cap = lambda x: min(x, 1)
        pairwise_bonferroni_corrected_p_values = (p_value_ij * bonferroni_correction_factor).applymap(cap)
        return table_p_value_bonferroni_corrected, pairwise_bonferroni_corrected_p_values
    
    def _test_for_single_pairwise_mutual_independence_using_rao_scott_2(self, observed):
        W = self.rows_factor.as_dataframe()
        Y = self.columns_factor.as_dataframe()
        I = self.rows_factor.level_count
        J = self.columns_factor.level_count
        spmi_df = pd.concat([W, Y], axis=1)  # type: pd.DataFrame

        def count_level_combinations(data, number_of_variables):
            data = data.copy()  # don't modify original dataframe
            level_arguments = [[0, 1] for i in range(0, number_of_variables)]
            variables = data.columns
            level_combinations = list(itertools.product(*level_arguments))
            full_combinations = pd.DataFrame(level_combinations, columns=variables)
            full_combinations["_dummy"] = 0
            data['_dummy'] = 1
            data = pd.concat([data, full_combinations]).reset_index(drop=True)
            grouped = data.groupby(list(variables))
            return grouped.sum().reset_index()

        W_count_ordered = count_level_combinations(W, I)
        Y_count_ordered = count_level_combinations(Y, J)
        n_count_ordered = count_level_combinations(spmi_df, I+J)

        n = len(spmi_df)
        G = (W_count_ordered.iloc[:, :-1]).T
        H = (Y_count_ordered.iloc[:, :-1]).T
        combined_counts = n_count_ordered.iloc[:, -1]
        tau = combined_counts / n
        m_row = G.dot(W_count_ordered.iloc[:, -1])
        m_col = H.dot(Y_count_ordered.iloc[:, -1])
        GH = np.kron(G, H)
        m = GH.dot(combined_counts)

        pi = m / n
        pi_row = m_row / n
        pi_col = m_col / n
        j_2r = np.ones((2 ** I, 1))
        i_2r = np.eye(2 ** I)
        j_2c = np.ones((2 ** J, 1))
        i_2c = np.eye(2 ** J)

        G_ij = G.dot(np.kron(i_2r, j_2c.T))
        H_ji = H.dot(np.kron(j_2r.T, i_2c))

        H_kron = np.kron(pi_row, H_ji.T).T  # extra .T's b/c Python handles vector/matrix kronecker differently than R
        G_kron = np.kron(G_ij.T, pi_col).T  # extra .T's b/c Python handles vector/matrix kronecker differently than R
        F = GH - H_kron - G_kron

        mult_cov = np.diag(tau) - np.outer(tau, tau.T)
        sigma = F.dot(mult_cov.dot(F.T))

        D = np.diag(np.kron(pi_row, pi_col) * np.kron(1 - pi_row, 1 - pi_col))
        Di_sigma = np.diag(1 / np.diag(D)).dot(sigma)
        eigenvalues, eigenvectors = linalg.eig(Di_sigma)
        Di_sigma_eigen = np.real(eigenvalues)
        sum_Di_sigma_eigen_sq = (Di_sigma_eigen ** 2).sum()

        observed_X_sq_S = observed.sum().sum()
        X_sq_S_rs2 = I * J * observed_X_sq_S / sum_Di_sigma_eigen_sq
        df_rs2 = (I ** 2) * (J ** 2) / sum_Di_sigma_eigen_sq
        X_sq_S_p_value_rs2 = chi2.sf(X_sq_S_rs2, df=df_rs2)
        return X_sq_S_p_value_rs2


class Factor(object):
    def __init__(self, data, labels, name, orientation="wide"):
        self.name = name
        self.labels = labels
        self.data = np.asarray(data, dtype=np.float64)
        self.orientation = orientation

    def __unicode__(self):
        return "Factor: {name}\nColumns:{columns}\nData:\n{data}".format(name=self.name, columns=self.labels,
                                                                         data=self.data)

    def __repr__(self):
        return self.__unicode__()

    def reshape_for_contingency_table(self):
        frame = self.as_dataframe()
        frame['observation_number'] = frame.index
        return pd.melt(frame, id_vars="observation_number")

    @property
    def level_count(self):
        return len(self.labels)

    def as_dataframe(self):
        return pd.DataFrame(self.data, columns=self.labels)

In [53]:
row_factor = Factor(data.data.iloc[:, :6], data.data.columns[:6], "expected_choice", orientation="wide")
column_factor = Factor(data.data.iloc[:, 6:11], data.data.columns[6:11], "believe_true", orientation="wide")
#crosstab_with_marginal_totals.fillna(0, inplace=True)  # fill value above doesn't seem to get all the na's

In [155]:
single_response_factor = Factor(data.data.iloc[:, :6], data.data.columns[:6], "expected_choice", orientation="wide")
multiple_response_factor = Factor(data.data.iloc[:, 6:11], data.data.columns[6:11], "believe_true", orientation="wide")

single_response_dataframe = single_response_factor.as_dataframe()
multiple_response_dataframe = multiple_response_factor.as_dataframe()

multiple_response_dataframe.head()

melted = pd.melt(single_response_dataframe.reset_index(), id_vars="index") \
    .rename(columns={"index": "response_id",
                     "value": "selected",
                     "variable": "single_response_level"})
melted = melted[melted.selected == 1]
single_response_melted = melted
multiple_response_dataframe.index.name = "response_id"
multiple_response_dataframe = multiple_response_dataframe.reset_index()
joint_dataframe = pd.merge(single_response_melted.iloc[:, :2], multiple_response_dataframe, how="inner", on="response_id")
joint_dataframe.head()

mmi_chi_squared_by_cell = pd.Series(index=multiple_response_factor.labels)
single_response_column = joint_dataframe.single_response_level

for c in multiple_response_factor.labels:
    multiple_response_column = joint_dataframe.loc[:, c]
    crosstab = pd.crosstab(single_response_column, multiple_response_column)
    chi2_results = chi2_contingency(crosstab, correction=False)
    chi_squared_statistic, table_p_value, degrees_of_freedom, expected_counts = chi2_results
    mmi_chi_squared_by_cell.loc[c] = chi_squared_statistic
mmi_chi_squared_by_cell

In [131]:
table = MRCVTable.from_data(data.data, 6, 5, shift_zeros=False)

In [132]:
observed = table._spmi_stat()

In [133]:
table._test_for_single_pairwise_mutual_independence_using_bonferroni(observed)
table._test_for_single_pairwise_mutual_independence_using_rao_scott_2(observed)

2.6696641484241839e-14

## Tests

In [181]:
from six import reraise

In [255]:

class MRCVTable(object):
    """
    Analyses that can be performed on a two-way contingency table that contains
    'multiple response' categorical variables (e.g. 'choose all that apply' questions).

    Parameters
    ----------
    table : array-like
        A contingency table.
    shift_zeros : boolean
        If True and any cell count is zero, add 0.5 to all values
        in the table.

    Attributes
    ----------
    table_orig : array-like
        The original table is cached as `table_orig`.
    marginal_probabilities : tuple of two ndarrays
        The estimated row and column marginal distributions.
    independence_probabilities : ndarray
        Estimated cell probabilities under row/column independence.

    See also
    --------
    statsmodels.graphics.mosaicplot.mosaic
    scipy.stats.chi2_contingency

    Notes
    -----


    References
    ----------
    Definitions of residuals:
        https://onlinecourses.science.psu.edu/stat504/node/86

    Bilder and Loughlin (2004)
    """

    def __init__(self, row_factors, column_factors, shift_zeros=True):

        self.row_factors = row_factors
        self.column_factors = column_factors
        self.table = self.table_from_factors(row_factors, column_factors)
        # TODO george@survata.com shift zeros

    def __unicode__(self):
        template = "Contingency Table With Multi-Response Categorical Variables (MRCV's).\nData:\n{table}"
        return template.format(table=self.table)

    def __repr__(self):
        return self.table.__repr__()

    @classmethod
    def from_data(cls, data, I, J,
                  rows_factor_name="factor_0", columns_factor_name="factor_1",
                  shift_zeros=True):
        """
        Construct a Table object from data.

        Parameters
        ----------
        data : array-like
            The raw data, from which a contingency table is constructed
            using the first two columns.
        shift_zeros : boolean
            If True and any cell count is zero, add 0.5 to all values
            in the table.
        I: The number of columns in the dataframe corresponding to the first factor
        J: The number of columns in the dataframe corresponding to the second factor

        Returns
        -------
        An MRCVTable instance.
        """

        if isinstance(data, pd.DataFrame):
            rows_data = data.iloc[:, 0:I]
            columns_data = data.iloc[:, I:I + J]
            rows_labels = rows_data.columns
            columns_labels = columns_data.columns
        else:
            rows_data = data[:, 0:I]
            columns_data = data[:, I:I + J]
            rows_labels = ["level_{}".format(i) for i in range(0, I)]
            columns_labels = ["level_{}".format(i) for i in range(I, I + J)]

        rows_factor = Factor(rows_data, labels=rows_labels, name=rows_factor_name, orientation="wide")
        columns_factor = Factor(columns_data, labels=columns_labels, name=columns_factor_name, orientation="wide")
        table = cls.table_from_factors([rows_factor], [columns_factor])
        return cls(table, rows_factor, columns_factor, shift_zeros)

    @classmethod
    def table_from_factors(cls, row_factors, column_factors):
        column_factor, row_factor = cls._extract_and_validate_factors(column_factors, row_factors)
        row_reshaped = row_factor.reshape_for_contingency_table()
        col_reshaped = column_factor.reshape_for_contingency_table()
        joint_dataframe = pd.merge(row_reshaped, col_reshaped, how="inner",
                                   on='observation_number', suffixes=("_row", "_col"))
        # without bool cast, '&' sometimes doesn't know how to compare types
        joint_response = joint_dataframe['value_row'].astype(bool) & joint_dataframe['value_col'].astype(bool)
        joint_dataframe['_joint_response'] = joint_response
        table = pd.pivot_table(joint_dataframe,
                               values='_joint_response',
                               fill_value=0,
                               index=['variable_row'],
                               columns=['variable_col'],
                               aggfunc=np.sum,)
        return table

    @classmethod
    def _extract_and_validate_factors(cls, column_factors, row_factors):
        try:
            try:
                if len(row_factors) > 1:
                    msg = "we don't currently support tables with more than one factor on the rows"
                    raise NotImplementedError(msg)
                row_factor = row_factors[0]
            except TypeError:
                if isinstance(row_factors, Factor):
                    row_factor = row_factors
                else:
                    msg = "row_factors must be either a list of Factors or a Factor instance"
                    raise NotImplementedError(msg)
            try:
                if len(column_factors) > 2:
                    msg = "we don't currently support tables with more than one factor on the columns"
                    raise NotImplementedError(msg)
                column_factor = column_factors[0]
            except TypeError as e:
                if isinstance(column_factors, Factor):
                    column_factor = column_factors
                else:
                    msg = "column_factors must be either a list of Factors or a Factor instance"
                    raise NotImplementedError(msg)
            return column_factor, row_factor
        except IndexError:
            explanation = ("Please be sure to pass at "
                           "least 1 factor on both the rows and columns")
            raise IndexError(explanation)

    def _calculate_pairwise_chi2s_for_MMI_item_response_table(self,
                                                              single_response_factor,
                                                              multiple_response_factor):
        """

        :param single_response_factor: 
        :param multiple_response_factor: 
        """
        single_response_dataframe = single_response_factor.as_dataframe()
        multiple_response_dataframe = multiple_response_factor.as_dataframe()

        single_response_melted = pd.melt(single_response_dataframe.reset_index(), id_vars="index") \
            .rename(columns={"index": "response_id",
                             "value": "selected",
                             "variable": "single_response_level"})
        single_response_melted = single_response_melted[single_response_melted.selected == 1]

        multiple_response_dataframe.index.name = "response_id"
        multiple_response_dataframe = multiple_response_dataframe.reset_index()
        joint_dataframe = pd.merge(single_response_melted.iloc[:, :2], multiple_response_dataframe, how="inner",
                                   on="response_id")

        mmi_chi_squared_by_cell = pd.Series(index=multiple_response_factor.labels)
        single_response_column = joint_dataframe.single_response_level
        for c in multiple_response_factor.labels:
            multiple_response_column = joint_dataframe.loc[:, c]
            crosstab = pd.crosstab(single_response_column, multiple_response_column)
            chi2_results = chi2_contingency(crosstab, correction=False)
            chi_squared_statistic, _, _, _ = chi2_results
            mmi_chi_squared_by_cell.loc[c] = chi_squared_statistic
        return mmi_chi_squared_by_cell

    def _calculate_pairwise_chi2s_for_SPMI_item_response_table(self, rows_factor, columns_factor):
        rows_data = rows_factor.data
        columns_data = columns_factor.data
        rows_levels = rows_factor.labels
        columns_levels = columns_factor.labels
        chis_spmi = pd.DataFrame(index=rows_levels, columns=columns_levels)
        for i in range(0, len(rows_levels)):
            for j in range(0, len(columns_levels)):
                rows = np.array(rows_data[:, i])
                columns = np.array(columns_data[:, j])
                row_name = rows_levels[i]
                col_name = columns_levels[j]
                crosstab = pd.crosstab(index=rows, columns=columns, rownames=[row_name], colnames=[col_name])
                chi2_results = chi2_contingency(crosstab, correction=False)
                chi_squared_statistic, _, _, _ = chi2_results
                chis_spmi.loc[row_name, col_name] = chi_squared_statistic
        return chis_spmi

    def _test_for_single_pairwise_mutual_independence_using_bonferroni(self, row_factor, column_factor):
        observed = self._calculate_pairwise_chi2s_for_SPMI_item_response_table(row_factor, 
                                                                               column_factor)
        chi2_survival_with_1_dof = partial(chi2.sf, df=1)
        p_value_ij = observed.applymap(chi2_survival_with_1_dof)
        p_value_min = p_value_ij.min().min()
        bonferroni_correction_factor = row_factor.level_count * column_factor.level_count
        table_p_value_bonferroni_corrected = bonferroni_correction_factor * p_value_min
        cap = lambda x: min(x, 1)
        pairwise_bonferroni_corrected_p_values = (p_value_ij * bonferroni_correction_factor).applymap(cap)
        return table_p_value_bonferroni_corrected, pairwise_bonferroni_corrected_p_values

    def _test_for_single_pairwise_mutual_independence_using_bootstrap(self, observed):
        W = self.row_factors.as_dataframe()
        Y = self.column_factors.as_dataframe()
        I = self.row_factors.level_count
        J = self.column_factors.level_count
        spmi_df = pd.concat([W, Y], axis=1)  # type: pd.DataFrame
        chi2_survival_with_1_dof = partial(chi2.sf, df=1)

        b_max = 1000
        n = len(spmi_df)
        q1 = spmi_df.iloc[:, :I + 1]
        q2 = spmi_df.iloc[:, I + 1:I + J]
        X_sq_S_star = []
        X_sq_S_ij_star = pd.DataFrame(index=range(0, I * J), columns=range(0, b_max))
        p_value_b_min = []
        p_value_b_prod = []
        rows_factor_name = self.row_factors.name
        rows_factor_labels = self.row_factors.labels
        columns_factor_name = self.column_factors.name
        columns_factor_labels = self.column_factors.labels
        for i in range(0, b_max):
            # pd.concat requires unique indexes...sampling with replacement produces duplicates
            q1_sample = q1.sample(n, replace=True).reset_index(drop=True)
            q2_sample = q2.sample(n, replace=True).reset_index(drop=True)

            sample_rows_factor = Factor(q1_sample, rows_factor_labels, rows_factor_name)
            sample_columns_factor = Factor(q2_sample, columns_factor_labels, columns_factor_name)
            stat_star = self._calculate_pairwise_chi2s_for_SPMI_item_response_table(sample_rows_factor,
                                                                                    sample_columns_factor)
            X_sq_S = stat_star.sum().sum()
            X_sq_S_star.append(X_sq_S)
            X_sq_S_ij_star.append(stat_star)
            p_value_ij = stat_star.applymap(chi2_survival_with_1_dof)
            p_value_min = p_value_ij.min().min()
            p_value_prod = p_value_ij.prod().prod()
            p_value_b_min.append(p_value_min)
            p_value_b_prod.append(p_value_prod)

        observed = self._calculate_pairwise_chi2s_for_SPMI_item_response_table(self.row_factors,
                                                                               self.column_factors)
        observed_X_sq_S = observed.sum().sum()
        p_value_ij = observed.applymap(chi2_survival_with_1_dof)
        p_value_min = p_value_ij.min().min()

        p_value_boot = np.mean(X_sq_S_star >= observed_X_sq_S)
        print(p_value_boot)

        p_value_boot_min_overall = np.mean(p_value_b_min <= p_value_min)
        print(p_value_boot_min_overall)

    def _test_for_single_pairwise_mutual_independence_using_rao_scott_2(self, row_factor,
                                                                        column_factor):
        observed = self._calculate_pairwise_chi2s_for_SPMI_item_response_table(row_factor, 
                                                                               column_factor)
        W = row_factor.as_dataframe()
        Y = column_factor.as_dataframe()
        I = row_factor.level_count
        J = column_factor.level_count
        spmi_df = pd.concat([W, Y], axis=1)  # type: pd.DataFrame

        def count_level_combinations(data, number_of_variables):
            data = data.copy()  # don't modify original dataframe
            level_arguments = [[0, 1] for i in range(0, number_of_variables)]
            variables = data.columns
            level_combinations = list(itertools.product(*level_arguments))
            full_combinations = pd.DataFrame(level_combinations, columns=variables)
            full_combinations["_dummy"] = 0
            data['_dummy'] = 1
            data = pd.concat([data, full_combinations]).reset_index(drop=True)
            grouped = data.groupby(list(variables))
            return grouped.sum().reset_index()

        W_count_ordered = count_level_combinations(W, I)
        Y_count_ordered = count_level_combinations(Y, J)
        n_count_ordered = count_level_combinations(spmi_df, I+J)

        n = len(spmi_df)
        G = (W_count_ordered.iloc[:, :-1]).T
        H = (Y_count_ordered.iloc[:, :-1]).T
        combined_counts = n_count_ordered.iloc[:, -1]
        tau = combined_counts / n
        m_row = G.dot(W_count_ordered.iloc[:, -1])
        m_col = H.dot(Y_count_ordered.iloc[:, -1])
        GH = np.kron(G, H)
        m = GH.dot(combined_counts)

        pi = m / n
        pi_row = m_row / n
        pi_col = m_col / n
        j_2r = np.ones((2 ** I, 1))
        i_2r = np.eye(2 ** I)
        j_2c = np.ones((2 ** J, 1))
        i_2c = np.eye(2 ** J)

        G_ij = G.dot(np.kron(i_2r, j_2c.T))
        H_ji = H.dot(np.kron(j_2r.T, i_2c))

        # extra .T's b/c Python handles vector/matrix kronecker differently than R
        H_kron = np.kron(pi_row, H_ji.T).T
        G_kron = np.kron(G_ij.T, pi_col).T
        F = GH - H_kron - G_kron

        mult_cov = np.diag(tau) - np.outer(tau, tau.T)
        sigma = F.dot(mult_cov.dot(F.T))

        D = np.diag(np.kron(pi_row, pi_col) * np.kron(1 - pi_row, 1 - pi_col))
        Di_sigma = np.diag(1 / np.diag(D)).dot(sigma)
        eigenvalues, eigenvectors = linalg.eig(Di_sigma)
        Di_sigma_eigen = np.real(eigenvalues)
        sum_Di_sigma_eigen_sq = (Di_sigma_eigen ** 2).sum()

        observed_X_sq_S = observed.sum().sum()
        X_sq_S_rs2 = I * J * observed_X_sq_S / sum_Di_sigma_eigen_sq
        df_rs2 = (I ** 2) * (J ** 2) / sum_Di_sigma_eigen_sq
        X_sq_S_p_value_rs2 = chi2.sf(X_sq_S_rs2, df=df_rs2)
        return X_sq_S_p_value_rs2

    def _test_for_marginal_mutual_independence_using_bonferroni_correction(self,
                                                                           single_response_factor,
                                                                           multiple_response_factor):
        mmi_pairwise_chis = self._calculate_pairwise_chi2s_for_MMI_item_response_table(single_response_factor,
                                                                                       multiple_response_factor)
        # TODO check
        c = len(multiple_response_factor.labels)
        r = len(multiple_response_factor.labels)

        chi2_survival_with_1_dof = partial(chi2.sf, df=(r - 1))

        p_value_ij = mmi_pairwise_chis.apply(chi2_survival_with_1_dof)
        p_value_min = p_value_ij.min()

        bonferroni_correction_factor = c
        table_p_value_bonferroni_corrected = min(bonferroni_correction_factor * p_value_min, 1)
        pairwise_bonferroni_corrected_p_values = (p_value_ij * bonferroni_correction_factor).apply(lambda x: min(x, 1))
        return table_p_value_bonferroni_corrected, pairwise_bonferroni_corrected_p_values

    def _test_for_marginal_mutual_independence_using_rao_scott_2(self,
                                                                 single_response_factor,
                                                                 multiple_response_factor):
        W = single_response_factor.as_dataframe()
        if not isinstance(W, pd.Series):
            W = W.iloc[:, 0]
        Y = multiple_response_factor.as_dataframe()
        n = len(W)
        I = 1  # single response variable must have exactly one column
        J = len(Y.columns)
        c = J
        r = len(W.unique())

        def conjoint_combinations(srcv, mrcv):
            number_of_variables = 1 + len(mrcv.columns)
            srcv = srcv.copy()  # don't modify original dataframe
            mrcv = mrcv.copy()  # don't modify original dataframe
            srcv.name = "srcv"
            srcv_level_arguments = srcv.unique()
            mrcv_level_arguments = [[0, 1] for i in range(0, number_of_variables - 1)]
            level_arguments = [list(srcv_level_arguments), ] + mrcv_level_arguments
            variables = ['srcv', ] + list(mrcv.columns)
            level_combinations = list(itertools.product(*level_arguments))
            full_combinations = pd.DataFrame(level_combinations, columns=variables)
            full_combinations["_dummy"] = 0
            data = pd.concat([srcv, mrcv], axis=1)
            data.srcv.value_counts()
            data['_dummy'] = 1
            data = pd.concat([data, full_combinations]).reset_index(drop=True)
            grouped = data.groupby(list(variables))
            result = grouped.sum().reset_index()
            return result

        def count_level_combinations(data, number_of_variables):
            data = data.copy()  # don't modify original dataframe
            level_arguments = [[0, 1] for i in range(0, number_of_variables)]
            variables = data.columns
            level_combinations = list(itertools.product(*level_arguments))
            full_combinations = pd.DataFrame(level_combinations, columns=variables)
            full_combinations["_dummy"] = 0
            data['_dummy'] = 1
            data = pd.concat([data, full_combinations]).reset_index(drop=True)
            grouped = data.groupby(list(variables))
            return grouped.sum().reset_index()

        Y_count_ordered = count_level_combinations(Y, J)
        n_count_ordered = conjoint_combinations(W, Y)
        Y_count_ordered.sort_values("_dummy")

        # need make n_iplus be in same order as SRCV options in the n_counts_ordered_table
        srcv_table_order = n_count_ordered.groupby('srcv').first().index.values
        n_iplus = W.value_counts().reindex(srcv_table_order)
        tau = n_count_ordered.iloc[:, -1].astype(int) / np.repeat(n_iplus, repeats=(2 ** c)).reset_index(drop=True)
        G_tilde = Y_count_ordered.iloc[:, :-1].T
        I_r = np.eye(r)
        G = np.kron(I_r, G_tilde)
        pi = G.dot(tau)
        m = pi * np.repeat(n_iplus, c)
        a_i = n_iplus / n
        pi_not_j = (1 / n) * np.kron(np.ones(r), np.eye(c)).dot(m)
        j_r = np.ones(r)
        pi_not = np.kron(j_r, pi_not_j)
        I_rc = np.eye(r * c)
        I_c = np.eye(c)
        J_rr = np.ones((r, r))
        A = np.diag(a_i)
        H = I_rc - np.kron(J_rr.dot(A), I_c)
        D = np.kron(np.diag(n / n_iplus), np.diag(pi_not_j) * (1 - pi_not_j))
        v_dim = r * (2 ** c)
        V = np.zeros((v_dim, v_dim))

        # TODO george@survata.com check starting 1 vs 0
        for i in range(1, r):
            a = ((i - 1) * (2 ** c) + 1) - 1
            b = ((i - 1) * (2 ** c) + (2 ** c)) - 1
            tau_range = tau[a:b]
            a_v = (1 / a_i[i - 1])
            tau_diag = np.diag(tau_range)
            tau_tcrossproduct = np.outer(tau_range, tau_range.T)
            v = a_v * (tau_diag - tau_tcrossproduct)
            V[a:b, a:b] = v

        D_diag = np.diag(1 / np.diag(D))
        D_diag.dot(H).dot(G)
        Di_HGVGH = D_diag.dot(H).dot(G).dot(V.dot(G.T).dot(H.T))
        eigenvalues, eigenvectors = np.linalg.eig(Di_HGVGH)
        Di_HGVGH_eigen = np.real(eigenvalues)
        sum_Di_HGVGH_eigen_sq = (Di_HGVGH_eigen ** 2).sum()
        observed = self._calculate_pairwise_chi2s_for_MMI_item_response_table(single_response_factor,
                                                                              multiple_response_factor)
        observed_X_sq = observed.sum()
        X_sq_S_rs2 = ((r - 1) * c) * observed_X_sq / sum_Di_HGVGH_eigen_sq
        df_rs2 = ((r - 1) ** 2) * (c ** 2) / sum_Di_HGVGH_eigen_sq
        X_sq_S_p_value_rs2 = chi2.sf(X_sq_S_rs2, df=df_rs2)
        return X_sq_S_p_value_rs2


In [176]:

class Factor(object):
    def __init__(self, data, labels, name, orientation="wide"):
        self.name = name
        self.labels = labels
        self.data = np.asarray(data, dtype=np.float64)
        self.orientation = orientation

    def __unicode__(self):
        return "Factor: {name}\nColumns:{columns}\nData:\n{data}".format(name=self.name, columns=self.labels,
                                                                         data=self.data)

    def __repr__(self):
        return self.__unicode__()

    def reshape_for_contingency_table(self):
        frame = self.as_dataframe()
        frame['observation_number'] = frame.index
        return pd.melt(frame, id_vars="observation_number")

    @property
    def level_count(self):
        return len(self.labels)

    def as_dataframe(self):
        return pd.DataFrame(self.data, columns=self.labels)


In [261]:
from numpy.testing import assert_allclose, assert_equal

def test_calculate_pairwise_chi2s_for_MMI_item_response_table():
    rows_factor = Factor(data.data.iloc[:, :6], data.data.columns[:6], "expected_choice", orientation="wide")
    columns_factor = Factor(data.data.iloc[:, 6:11], data.data.columns[6:11], "believe_true", orientation="wide")
    multiple_response_table = MRCVTable([rows_factor,], [columns_factor])
    pairwise_chis = multiple_response_table._calculate_pairwise_chi2s_for_MMI_item_response_table(rows_factor, columns_factor)
    expected = np.array([ 27.49359188,  36.51100003,  33.06527619,  19.30508587,  16.2387889 ])
    assert_allclose(pairwise_chis.values, expected)
    
def test_calculate_pairwise_chi2s_for_SPMI_item_response_table():    
    rows_factor = Factor(data.data.iloc[:, 11:], data.data.columns[11:], "reasons_undecided", orientation="wide")
    columns_factor = Factor(data.data.iloc[:, 6:11], data.data.columns[6:11], "believe_true", orientation="wide")
    multiple_response_table = MRCVTable([rows_factor,], [columns_factor])
    pairwise_chis = multiple_response_table._calculate_pairwise_chi2s_for_SPMI_item_response_table(rows_factor, columns_factor)
    expected = np.array([[11.057399043055453, 7.0433141769624967, 11.76419998565429,
            6.6835142302920527, 16.129398885445724],
           [12.510475178886146, 0.0031863099132473853, 0.5586384786490618,
            2.0740299456382045, 0.96428862745228061],
           [17.27417447368591, 2.7434061504889233, 9.7400222125093734,
            10.013109401042946, 22.869451447577219],
           [0.022869776364994012, 1.3818184772649058, 0.4021032851909711,
            0.019325630680345859, 8.5606054391027779],
           [5.2158331412745191, 4.0136842422000854, 16.088255726022293,
            4.4883332473823732, 0.23695713171009866]], dtype=np.float64)
    observed = pairwise_chis.values.astype(np.float64)
    assert_allclose(observed, expected)

def test_multiple_mutual_independence_true():
    assert False

def test_multiple_mutual_independence_true_using_bonferroni():
    rows_factor = Factor(data.data.iloc[:, :6], data.data.columns[:6], "expected_choice", orientation="wide")
    columns_factor = Factor(data.data.iloc[:, 6:11], data.data.columns[6:11], "believe_true", orientation="wide")
    multiple_response_table = MRCVTable([rows_factor,], [columns_factor])
    table_p_value, cellwise_p_values = multiple_response_table._test_for_marginal_mutual_independence_using_bonferroni_correction(rows_factor, columns_factor)
    assert table_p_value - 1.1356954469547448e-06 <= 0.00001
    expected = np.array([7.89787134e-05, 1.13569545e-06, 5.79140286e-06,
             3.42276565e-03, 1.35746237e-02])
    assert_allclose(cellwise_p_values, expected)

def test_multiple_mutual_independence_true_using_rao_scott_2():
    rows_factor = Factor(data.data.iloc[:, :6], data.data.columns[:6], "expected_choice", orientation="wide")
    columns_factor = Factor(data.data.iloc[:, 6:11], data.data.columns[6:11], "believe_true", orientation="wide")
    multiple_response_table = MRCVTable([rows_factor,], [columns_factor])
    table_p_value = multiple_response_table._test_for_marginal_mutual_independence_using_rao_scott_2(rows_factor, columns_factor)
    assert table_p_value - 0.0 <= 0.00001

def test_test_multiple_mutual_independence_false():
    assert False

def test_spmi_true_using_bonferroni():
    rows_factor = Factor(data.data.iloc[:, 11:], data.data.columns[11:], "reasons_undecided", orientation="wide")
    columns_factor = Factor(data.data.iloc[:, 6:11], data.data.columns[6:11], "believe_true", orientation="wide")
    multiple_response_table = MRCVTable([rows_factor,], [columns_factor])
    table_p_value, cellwise_p_values = multiple_response_table._test_for_single_pairwise_mutual_independence_using_bonferroni(rows_factor, columns_factor)
    assert table_p_value - 4.3346430242129665e-05 <= 0.000001
    expected = np.array([[  2.20834933e-02,   1.98904185e-01,   1.50952071e-02,
              2.43271486e-01,   1.47896365e-03],
           [  1.01169167e-02,   1.00000000e+00,   1.00000000e+00,
              1.00000000e+00,   1.00000000e+00],
           [  8.08886522e-04,   1.00000000e+00,   4.50746797e-02,
              3.88574715e-02,   4.33464302e-05],
           [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
              1.00000000e+00,   8.58787906e-02],
           [  5.59553004e-01,   1.00000000e+00,   1.51144522e-03,
              8.53173376e-01,   1.00000000e+00]])
    assert_allclose(cellwise_p_values.values, expected)


def test_spmi_true_using_rao_scott_2():
    rows_factor = Factor(data.data.iloc[:, 11:], data.data.columns[11:], "reasons_undecided", orientation="wide")
    columns_factor = Factor(data.data.iloc[:, 6:11], data.data.columns[6:11], "believe_true", orientation="wide")
    multiple_response_table = MRCVTable([rows_factor,], [columns_factor])
    table_p_value = multiple_response_table._test_for_single_pairwise_mutual_independence_using_rao_scott_2(rows_factor, columns_factor)
    assert table_p_value - 6.2565046672587634e-18 <= 0.000001
    
def test_single_pairwise_mutual_independence_true():
    assert False


def test_single_pairwise_mutual_independence_false():
    assert False


def test_MRCV_table_from_data():
    assert False


def test_MRCV_table_from_factors():
    assert False


def test_Factor_from_wide_data():
    assert False


def test_Factor_from_narrow_data():
    assert False

In [262]:
test_calculate_pairwise_chi2s_for_MMI_item_response_table()
test_multiple_mutual_independence_true_using_bonferroni()
test_multiple_mutual_independence_true_using_rao_scott_2()
test_calculate_pairwise_chi2s_for_SPMI_item_response_table()
test_spmi_true_using_bonferroni()
test_spmi_true_using_rao_scott_2()

In [171]:
import statsmodels.api as sm
from statsmodels.datasets import presidential2016

data = sm.datasets.presidential2016.load_pandas()
data.data.head()

,Hillary_Clinton,Donald_Trump,Jill_Stein,Gary_Johnson,None_Of_The_Above,I_Probably_Wont_Vote,Hillary_Clinton_is_involved_in_many_coverups,Trump_changes_his_positions_all_of_the_time,Hillary_Clinton_lied_to_the_families_of_Americans_killed_in_Benghazi,Trump_is_a_successful_businessman,Trumps_temper_could_get_the_country_into_trouble,I_wish_another_candidate_had_won_the_primary,Need_to_do_more_research,Dont_like__any_candidate,Not_sure_which_candidate_shares_my_values,Waiting_for_debates
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [258]:
rows_factor = Factor(data.data.iloc[:, 11:], data.data.columns[11:], "reasons_undecided", orientation="wide")
columns_factor = Factor(data.data.iloc[:, 6:11], data.data.columns[6:11], "believe_true", orientation="wide")
multiple_response_table = MRCVTable([rows_factor,], [columns_factor])
table_p_value = multiple_response_table._test_for_single_pairwise_mutual_independence_using_rao_scott_2(rows_factor, columns_factor)
assert table_p_value - 6.2565046672587634e-18 <= 0.000001


6.2565046672587634e-18

In [239]:
assert_allclose(table_p_value.values.astype(np.float64), expected)

In [252]:
cellwise_p_values.values

array([[  2.20834933e-02,   1.98904185e-01,   1.50952071e-02,
          2.43271486e-01,   1.47896365e-03],
       [  1.01169167e-02,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.00000000e+00],
       [  8.08886522e-04,   1.00000000e+00,   4.50746797e-02,
          3.88574715e-02,   4.33464302e-05],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   8.58787906e-02],
       [  5.59553004e-01,   1.00000000e+00,   1.51144522e-03,
          8.53173376e-01,   1.00000000e+00]])